In [2]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts

/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/antropy/fractal.py:197: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit((types.Array(types.float64, 1, "C", readonly=True), types.int32))
/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package yasa is out of date. Your version is 0.6.3, the latest is 0.6.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [4]:
t1 = time.time()

channels = constants.channel_list
data_types = ['N2', 'N3','REM', 'Wake', 'N1']
data_type = 'Wake' #Start with Wake data as it's easiest to process, do the others afterwards

# 1. generate all path names and class list(s) etc. 
folder = '/user/home/ko20929/work/RBD_using_custom_package/Blue_pebble/'
paths = joblib.load(folder + data_type + '_paths.pkl') # keys : ['selected_paths', 's_class_list', 's_night_list', 's_sleep_type', 's_p_id']

# 2. Load corresponding data into dataframes, store in dataframe list
df_list = []
error_paths = []
with io.capture_output() as captured:
    for path in paths['selected_paths']:
        data_epo = mne.read_epochs(path)
        data = data_epo._data * 1e6  # convert signal from V to uV
        df_full = data_epo.to_data_frame()
        df = df_full[channels].copy()
        df_list.append(df)

t2 = time.time()
t2-t1

6.43300986289978

#### TS version of processing the data (don't need to run)

In [83]:
t1 = time.time()

ts_row_list = []

for df in df_list[:10]:
    #1.Generate the window indices 
    window_indices = eeg_stat_ts.gen_window_indices(6, df , samp_freq = 256)
    #2. Calculate bpw vals per window
    bpw_per_win_df = eeg_stat_ts.gen_statistic_per_window(df , window_indices , stat = 'bpw')
    #3. Convert bpw per window per channel into bpw per window per region
    regional_df = eeg_stat_ts.convert_chan_stats_to_region(bpw_per_win_df, constants.channel_list , constants.region_to_channel_dict)
    #4. Convert into a single row of a new dataframe where each cell is a series
    new_row = eeg_stat_ts.dataframe_to_ts_row(regional_df, list(regional_df.columns[:-1]) )
    ts_row_list.append(new_row)

t2 = time.time()

t2-t1

4.5045201778411865

In [66]:
ts_df = pd.DataFrame.from_records(ts_row_list)
ts_df.head(1)

,Delta_Central,Theta_Central,Alpha_Central,Sigma_Central,Beta_Central,Gamma_Central,TotalAbsPow_Central,Delta_Frontal,Theta_Frontal,Alpha_Frontal,...,Beta_Right Parietal,Gamma_Right Parietal,TotalAbsPow_Right Parietal,Delta_Right Temporal,Theta_Right Temporal,Alpha_Right Temporal,Sigma_Right Temporal,Beta_Right Temporal,Gamma_Right Temporal,TotalAbsPow_Right Temporal
0,0 0.119395 1 0.115076 2 0.095208 3...,0 0.305156 1 0.240114 2 0.469296 3...,0 0.274594 1 0.358792 2 0.177561 3...,0 0.126387 1 0.111546 2 0.095823 3...,0 0.138452 1 0.143712 2 0.146880 3...,0 0.036015 1 0.030761 2 0.015232 3...,0 49.601829 1 46.612778 2 68.77758...,0 0.078060 1 0.081185 2 0.051734 3...,0 0.283393 1 0.227771 2 0.525861 3...,0 0.293643 1 0.499218 2 0.228874 3...,...,0 0.085936 1 0.097931 2 0.062913 3...,0 0.038117 1 0.043186 2 0.026132 3...,0 75.495552 1 69.002663 2 111.40...,0 0.247600 1 0.190804 2 0.163329 3...,0 0.294811 1 0.448443 2 0.541493 3...,0 0.196601 1 0.183503 2 0.127083 3...,0 0.085892 1 0.056702 2 0.063474 3...,0 0.125665 1 0.080969 2 0.084591 3...,0 0.049432 1 0.039578 2 0.020029 3...,0 57.018125 1 66.747889 2 73.49080...


#### Non TS version of processing the data

In [5]:
t1 = time.time()

ts_row_list = []

for df in df_list:
    #1. Calculate bpw values
    bpw_df = eeg_stat_ts.gen_band_power_vals_and_freq_ratios(df, list(df.columns), win_s = 8)    
    bpw_df['window_no.'] = 0
    #2. Convert bpw per channel into bpw per region
    regional_df = eeg_stat_ts.convert_chan_stats_to_region(bpw_df, constants.channel_list , constants.region_to_channel_dict)
    #4. Convert into a single row of a new dataframe where each cell is a series
    new_row = eeg_stat_ts.dataframe_to_ts_row(regional_df, list(regional_df.columns[:-1]), time_steps = False )
    ts_row_list.append(new_row)
    
ts_df = pd.DataFrame.from_records(ts_row_list)
groups = pd.Series(paths['s_p_id'])
s_class_list = pd.Series(paths['s_class_list'])
y = s_class_list.map({'HC': 0 , 'PD' : 1 , 'PD+RBD' : 2 , 'RBD' : 3}) 

t2 = time.time()
t2-t1

12.071491956710815

In [6]:
ts_df

,Delta_Central,Theta_Central,Low_Alpha_Central,Alpha_Central,High_Alpha_Central,Sigma_Central,Beta_Central,Gamma_Central,TotalAbsPow_Central,Fast_ratio_Central,...,Theta_Right Temporal,Low_Alpha_Right Temporal,Alpha_Right Temporal,High_Alpha_Right Temporal,Sigma_Right Temporal,Beta_Right Temporal,Gamma_Right Temporal,TotalAbsPow_Right Temporal,Fast_ratio_Right Temporal,Alpha_shift_Right Temporal
0,0.133384,0.363460,0.385395,0.207255,0.064340,0.114314,0.153863,0.027724,42.522282,0.736820,...,0.488953,0.505140,0.154827,0.034874,0.073245,0.096049,0.027527,56.171060,0.394989,14.766051
1,0.212296,0.584565,0.562044,0.093433,0.013762,0.048039,0.053650,0.008016,28.897585,0.186093,...,0.484217,0.482427,0.113761,0.014012,0.044779,0.057873,0.021959,37.765458,0.228936,34.563716
2,0.328396,0.184740,0.207937,0.297405,0.049565,0.056721,0.116740,0.015997,29.146105,0.811704,...,0.123573,0.162756,0.198046,0.029687,0.053988,0.092805,0.028206,83.824821,0.512036,5.548911
3,0.151747,0.085880,0.103761,0.501193,0.075629,0.079254,0.169988,0.011937,44.529488,3.086845,...,0.098183,0.110420,0.401529,0.068181,0.084082,0.132667,0.039460,47.694334,1.717654,1.732292
4,0.172530,0.096115,0.153947,0.266387,0.048406,0.146355,0.287883,0.030729,25.143283,2.135589,...,0.079444,0.105202,0.199726,0.042646,0.092698,0.267035,0.205922,47.782966,2.228873,2.510016
5,0.274479,0.394292,0.409523,0.154654,0.023834,0.084924,0.077283,0.014368,22.163283,0.355375,...,0.354807,0.366625,0.139761,0.021914,0.079840,0.149512,0.094279,31.621821,0.671217,16.364740
6,0.389097,0.116812,0.077706,0.108526,0.029763,0.117721,0.230634,0.037210,10.999173,0.679483,...,0.079340,0.067790,0.132464,0.036902,0.098125,0.165939,0.065684,16.428195,0.565055,1.856562
7,0.337400,0.111216,0.088046,0.158878,0.042584,0.072463,0.299252,0.020791,13.916875,1.035114,...,0.089082,0.090783,0.206547,0.049766,0.092162,0.215108,0.067805,18.693057,1.066902,1.821566
8,0.448429,0.177922,0.168330,0.169116,0.027791,0.052729,0.131557,0.020246,33.258316,0.499223,...,0.186824,0.202970,0.185504,0.028076,0.082214,0.165258,0.047048,50.650400,0.720575,7.369414
9,0.433525,0.142319,0.149223,0.249432,0.053514,0.043641,0.105679,0.025403,14.071549,0.621563,...,0.114150,0.163346,0.304325,0.063002,0.044508,0.093454,0.036987,26.562156,0.806567,2.769049


In [11]:
joblib.dump(ts_df, 'Wake_frequency_data.pkl')

['Wake_frequency_data.pkl']